In [2]:
import cv2
import numpy as np
import os

In [2]:
os.environ['OPENCV_VIDEOIO_PRIORITY_MSMF'] = "0"

In [3]:
#%% Template Matching

source = cv2.imread("./sources/source.jpg")
template = cv2.imread("./sources/template.jpg")

source_gray = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

w,h = template_gray.shape[::-1] # bütün herşeyi al, ama -1 olduğu için tam tersini al
# template_gray.shape => 80,120
R = cv2.matchTemplate(source_gray, template_gray, cv2.TM_CCOEFF_NORMED)

threshold = 0.6

loc = np.where(R > threshold) # => R ı threshold dan büyük değerleri al
# loc => koordinatları
for pt in zip(*loc[::-1]): # zip(*loc[::-1]) => x ve y yi birleştiriyoruz.
    cv2.rectangle(source, pt, (pt[0]+w, pt[1]+h), (255,120,15), 2)

cv2.imshow("Source", source)
cv2.imshow("Template", template)
#cv2.imshow("R",R)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [10]:
#%% viola-jones algoritması

img = cv2.imread("./sources/evren.png",0) # 0 yazarak img yi direk gray e çeviriyoruz.

face_xml = cv2.CascadeClassifier("./sources/haarcascade_frontalface_default.xml")
# frontal xml dosyasınıda aldık
result = face_xml.detectMultiScale(img,1.3, 4) # img,1.3, 4 => resmi 4 kere tara 1.3 oranında büyüt.
# result => [[157 148 219 219]]
# len(result[0]) => 4
x, y, w, h = [result[0][i] for i in range(len(result[0]))] #List comprehension
# x, y => çizeceğimiz dikdörtgenin en üst sol pikselleri
# w, h => çizeceğimiz dikdörtgenin genişlik ve yüksekliği
# x = 157, y = 148, w = 219, h = 219
cv2.rectangle(img, (x,y), (x+w,y+h), (255,255,0),3)
# (x,y) => sol üst
# (x+w,y+h) => sağ alt
cv2.imshow("Evren",img)

cv2.waitKey(0)
cv2.destroyAllWindows()

4


In [14]:
#%% Yuz tespiti toplu

img = cv2.imread("./sources/crowd.jpg")

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

face_xml = cv2.CascadeClassifier("./sources/haarcascade_frontalface_default.xml")

result2 = face_xml.detectMultiScale(img_gray,1.1, 1)


for x,y,w,h in result2:
    cv2.rectangle(img, (x,y), (x+w,y+h), (255,255,0), 3)

cv2.imshow("Crowded",img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
#%% Anlık görüntü yüz tespiti

cam = cv2.VideoCapture(0)

face_xml = cv2.CascadeClassifier("./sources/haarcascade_frontalface_default.xml")

while(cam.isOpened()):
    ret,frame = cam.read()
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    my_result = face_xml.detectMultiScale(frame_gray, 1.1,2)
    
    for x,y,w,h in my_result:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 3)
        
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
cam.release()
cv2.destroyAllWindows()

In [3]:
#%% Anlık görüntü yüz ve göz tespiti

cam = cv2.VideoCapture(0)
face_xml = cv2.CascadeClassifier("./sources/haarcascade_frontalface_default.xml")
eyes_xml = cv2.CascadeClassifier("./sources/haarcascade_eye.xml")

while(cam.isOpened()):
    
    ret,frame = cam.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    my_result = face_xml.detectMultiScale(frame_gray, 1.3,3)


    for x,y,w,h in my_result:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 3)
        # Buradan sonra gözleri bulalım
        face_roi = frame[y:y+h, x:x+w] # yüzümüzün roi si yani yüzümüzü aldık
        face_roi_gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY) # gray e çevirdik
        eyes_result = eyes_xml.detectMultiScale(face_roi_gray)# gözleri bulduk
        for x_2, y_2, w_2, h_2 in eyes_result:
            cv2.rectangle(face_roi, (x_2,y_2), (x_2+ w_2, y_2 + h_2), (255,0,0),3)      
        
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
cam.release()
cv2.destroyAllWindows()

In [18]:
#%% Full Body

cam = cv2.VideoCapture("./sources/people2.mp4")
body_xml = cv2.CascadeClassifier("./sources/haarcascade_fullbody.xml")

while(cam.isOpened):
    ret,frame = cam.read()
    if ret != True:
        print("Video is over")
        break
    frame= cv2.resize(frame,(600,400))
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    my_result = body_xml.detectMultiScale(frame_gray, 1.1,1)
    
    for x_2, y_2, w_2, h_2 in my_result:
        cv2.rectangle(frame, (x_2,y_2), (x_2+ w_2, y_2 + h_2), (0,255,255),3)
    
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
cam.release()
cv2.destroyAllWindows()

In [8]:
#%% Plate Recognition (plaka)

plate = cv2.imread("./sources/plate4.jpg")

plate_gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
plate_xml = cv2.CascadeClassifier("./sources/haarcascade_russian_plate_number.xml")

result2 = plate_xml.detectMultiScale(plate_gray,1.1,4)

for x,y,w,h in result2:
    cv2.rectangle(plate, (x,y), (x+w,y+h), (0,255,255),3)
    
cv2.imshow("Plate", plate)

cv2.waitKey(0)
cv2.destroyAllWindows()